# Simulation of turing pattern in 1D

The systeme of reaction-diffusion is the following one :

$$
\left\{
\begin{array}{l}
\partial_t u =  D_u\, \partial^2 _x u + f(u,v), 
\quad \displaystyle f(u,v) = a - u - \frac{4\,u\,v}{1+u^2},\\
\partial_t v = \delta\left\lbrack D_v \, \partial^2 _x v  + g(u,v)\right\rbrack,
\quad \displaystyle g(u,v) = b \left(u - \frac{u\,v}{1+u^2}\right)
\end{array}
\right.
$$

In [ ]:
import numpy as np

from scipy.integrate import solve_ivp

import random

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.palettes import Category10, Category20
from bokeh.models import PrintfTickFormatter

from ipywidgets import FloatProgress, IntProgress
from IPython.display import display

from mylib.turing_model import turing_model

import mylib.integration as integration

output_notebook(hide_banner=True)

In [ ]:
def plot_sol():

    a = 30.
    b = 2.8
    eps = 0.02
    xmin = 0.
    xmax = 50.
    # nb of points including boundary conditions
    nxib = 1001
    nx = nxib-2
    tini = 0.0
    tend = 600.0
    neq = 2
    
    progress_bar = FloatProgress(min=tini, max=tend, value=tini, description='Progress:')
    display(progress_bar)
    
    tm = turing_model(a=a, b=b, xmin=xmin, xmax=xmax, nx=nx)
    fcn = tm.fcn_radau

    yini = np.zeros((nx, neq))
    for inx in range(nx):
        yini[inx, 0] = a/5 + (1-2*random.random())*eps
        yini[inx, 1] = 1 + (a*a)/25 + (1-2*random.random())*eps
    yini = yini.flatten()
    
    x = np.linspace(xmin, xmax, nx)
    fig_sol_u = figure(x_range=(xmin, xmax), y_range=(0,11), plot_height=300, plot_width=950, title="Solution u")
    fig_sol_v = figure(x_range=(xmin, xmax), y_range=(19,45), plot_height=300, plot_width=950, title="Solution v")

    u = yini.reshape(nx,neq)[:,0]
    v = yini.reshape(nx,neq)[:,1]
    plt_sol_u = fig_sol_u.x(x, u)
    plt_sol_v = fig_sol_v.x(x, v)
    show(column(fig_sol_u, fig_sol_v), notebook_handle=True)
    
    nt_plot = 11
    t_plot = np.linspace(tini, tend, nt_plot)
    
    for it, ti in enumerate(t_plot[:-1]):
        
        ##print(it)
        sol = integration.radau5(t_plot[it], t_plot[it+1], yini, fcn, njac=2, atol=1.e-8, rtol=1.e-8)
        u = sol.y.reshape(nx,neq)[:,0]
        v = sol.y.reshape(nx,neq)[:,1]

        plt_sol_u.data_source.data = dict(x=x, y=u)
        plt_sol_v.data_source.data = dict(x=x, y=v)
        fig_sol_u.title.text = f"sol u at t = {t_plot[it+1]}"
        fig_sol_v.title.text = f"sol v at t = {t_plot[it+1]}"
        push_notebook()
        
        yini = sol.y
        progress_bar.value = t_plot[it+1]

plot_sol()

In [ ]:
def plot_sol_bis():

    a = 30.
    b = 2.8
    eps = 0.02
    xmin = 0.
    xmax = 50.
    # nb of points including boundary conditions
    nxib = 1001
    nx = nxib-2
    tini = 0.0
    tend = 600.0
    neq = 2
    
    progress_bar = FloatProgress(min=tini, max=tend, value=tini, description='Progress:')
    display(progress_bar)
 
    tm = turing_model(a=a, b=b, xmin=xmin, xmax=xmax, nx=nx)
    fcn = tm.fcn_radau

    yini = np.zeros((nx, neq))
    for inx in range(nx):
        yini[inx, 0] = a/5 + (1-2*random.random())*eps
        yini[inx, 1] = 1 + (a*a)/25 + (1-2*random.random())*eps
    yini = yini.flatten()
    
    dx = (xmax-xmin)/(nx-1)
    x = np.linspace(xmin, xmax, nx)
    fig_sol_u = figure(x_range=(xmin, xmax), plot_height=300, plot_width=900, title="Solution u")
    fig_sol_v = figure(x_range=(xmin, xmax),plot_height=300, plot_width=900, title="Solution v")
    u = yini.reshape(nx,neq)[:,0]
    v = yini.reshape(nx,neq)[:,1]
    plt_sol_u = fig_sol_u.x(x, u, color="Grey")
    plt_sol_v = fig_sol_v.x(x, v, color="Grey")
    show(column(fig_sol_u, fig_sol_v), notebook_handle=True)
    
    nt_plot = 11
    t_plot = np.linspace(tini, tend, nt_plot)
    
    for it, ti in enumerate(t_plot[:-1]):
        
        sol = integration.radau5(t_plot[it], t_plot[it+1], yini, fcn, njac=2, atol=1.e-8, rtol=1.e-8)
        u = sol.y.reshape(nx,neq)[:,0]
        v = sol.y.reshape(nx,neq)[:,1]

        fig_sol_u.x(x, u, color=Category20[20][it])
        fig_sol_v.x(x, v, color=Category20[20][it])

        fig_sol_u.title.text = f"sol u at t = {t_plot[it+1]}"
        fig_sol_v.title.text = f"sol u at t = {t_plot[it+1]}"

        push_notebook()
        
        yini = sol.y
        progress_bar.value = t_plot[it+1]

            
plot_sol_bis()